In [29]:
# 1. Importação de Bibliotecas e Preparação do Ambiente

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import json
import string

# Download de recursos da NLTK (necessário apenas na primeira execução)
try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')

#nltk.download('rslp')

In [42]:
with open(r'JsonSoup/JsonRacas/stem/dwarf_stemmed_structured.json', 'r') as file_object:
        jsonCarregado = json.load(file_object)

def extract_strings(obj):
    if isinstance(obj, str):
        yield obj
    elif isinstance(obj, dict):
        for v in obj.values():
            yield from extract_strings(v)
    elif isinstance(obj, list):
        for item in obj:
            yield from extract_strings(item)

# Turn into a list (corpus)
corpusAux = list(extract_strings(jsonCarregado))

corpus = [
    s.translate(str.maketrans("", "", string.punctuation))
    for s in corpusAux
]
query = 'the artificer casts spells'

stemmer = PorterStemmer()
# Carrega a lista de stopwords original da NLTK
english_stop_words_original = set(stopwords.words('english'))
english_stop_words_original.update(string.punctuation)

stop_words_stemmed = [stemmer.stem(word) for word in english_stop_words_original]

def stem_tokenizer(text):
    tokens = text.split()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return stemmed_tokens

bow_vec = CountVectorizer(
    lowercase=True,
    stop_words=stop_words_stemmed,  # Usa a lista de stopwords corrigida
    tokenizer=stem_tokenizer
)
X_bow = bow_vec.fit_transform(corpus)
df_bow = pd.DataFrame(X_bow.toarray(), columns=bow_vec.get_feature_names_out())

print("Matriz Documento-Termo (BoW):")
df_bow



{'him', 'under', 'hasn', 'of', 'shan', '(', 'yourself', ';', "it's", 'such', 'we', 'hadn', "we'd", '@', 'does', 'where', 'for', 'in', 've', 'me', '%', 'ain', 'm', 'which', '?', 'both', 'more', "needn't", 'why', 'theirs', '*', 'am', 'through', 'hers', 'its', 'then', 'during', 'been', 'mightn', 'can', 'above', 'don', "mustn't", 'you', "you've", '}', 'shouldn', 'should', 'doing', ']', 'those', 'his', "he's", 'won', "you'd", 'it', "she's", 'ma', 'against', 'couldn', '!', 'being', 'have', "wouldn't", "aren't", 'be', '~', 'my', 'between', '|', 'mustn', 't', 'than', 'needn', 'that', "you'll", "they'll", '-', 'herself', 'doesn', 'wasn', 'after', 'not', '>', '[', 'and', '\\', 'no', 'has', 'into', 'very', "won't", 'a', 'the', 'off', 'if', 'how', 'o', "they've", 'few', 'too', "mightn't", 'up', '&', "hadn't", "isn't", "they'd", 'had', "it'll", '+', 'do', "i've", 'out', 'about', 'to', '$', "she'd", "she'll", "you're", "it'd", ')', 'below', 'again', 'having', "we'll", "shan't", "shouldn't", 'is', 'o

C:\Users\deant\Documentos\FURB\PLN\WebScrapping\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\deant\Documentos\FURB\PLN\WebScrapping\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['becau'] not in stop_words.
  warnings.warn(


,1,150,2,25,350,4,5,50,60,abil,...,well,wellord,whatev,whenev,wisdom,within,write,year,young,–
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0
4,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
5,0,1,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
